In [5]:
import os

In [2]:
from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')
data_dir = "/projects/home/mmasood1/arslan_data_repository/DILI"

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('DILI') 
    # all benchmark names in a benchmark group are stored in group.dataset_names
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
    seed_dir = data_dir + f"/seed_{seed}/"
    os.makedirs(seed_dir, exist_ok = True)
    train.to_csv(seed_dir + f"train_seed_{seed}.csv", index = False)
    valid.to_csv(seed_dir + f"valid_seed_{seed}.csv", index = False)
    test.to_csv(seed_dir + f"test_seed_{seed}.csv", index = False)

/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/mmasood1/.conda/envs/env_arslan/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
Found local copy...
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 2094.73it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 2110.23it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 2074.94it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 2071.13it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 2068.66it/

# SMILES Normalization

In [6]:
import pandas as pd
import math
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize
#IPythonConsole.drawOptions.comicMode=True
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.info')
import rdkit
from rdkit.Chem.SaltRemover import SaltRemover
print(rdkit.__version__)
import numpy as np

from multiprocessing import Pool
from tqdm import tqdm

2023.03.3


In [7]:
def standardize(smiles,remover=SaltRemover()):
    config = {}
    config["StandardizeSmiles"] = True
    config["FragmentParent"] = False
    config["SaltRemover"] = True
    config["isomericSmiles"] = False
    config["kekuleSmiles"] = True
    config["canonical"] = True
    # follows the steps in
    # https://github.com/rdkit/rdkit/blob/master/Docs/Notebooks/MolStandardize.ipynb
    try:
        if config["StandardizeSmiles"]:
            # removeHs, disconnect metal atoms, normalize the molecule, reionize the molecule

                smiles = rdMolStandardize.StandardizeSmiles(smiles)

        mol = Chem.MolFromSmiles(smiles)
        # remove salts
        if config["SaltRemover"]:
            mol = remover.StripMol(mol, dontRemoveEverything=False) 

        if config["FragmentParent"]:
            mol = rdMolStandardize.FragmentParent(mol) 

        if config["kekuleSmiles"]:
            Chem.Kekulize(mol, clearAromaticFlags=True)
        normalized_smiles = Chem.MolToSmiles(mol, 
                            isomericSmiles = config["isomericSmiles"],
                            kekuleSmiles = config["kekuleSmiles"],
                            canonical = config["canonical"],
                            allHsExplicit = False)
        if normalized_smiles == '':
            normalized_smiles = np.nan
    except:
        normalized_smiles = np.nan
    return normalized_smiles

def normalize_smiles_parallel(smiles_list):
    with Pool() as pool:
        results = []
        total = len(smiles_list)
        with tqdm(total=total, ncols=80, desc="Processing") as pbar:
            for normalized_smiles in pool.imap(standardize, smiles_list):
                results.append(normalized_smiles)
                pbar.update(1)
    return results

In [8]:
data_dir = "/projects/home/mmasood1/arslan_data_repository/DILI"
dataset_list = ["train","valid","test"]

for seed in [1, 2, 3, 4, 5]:
    for dataset in dataset_list:
        seed_dir = data_dir + f"/seed_{seed}/"
        normlization_dir = seed_dir + "normalized_data/"
        os.makedirs(normlization_dir, exist_ok = True)

        data = pd.read_csv(seed_dir + f"{dataset}_seed_{seed}.csv")
        total_mol = data.shape[0]

        data.drop("Drug_ID", axis = 1, inplace = True)
        data.rename(columns= {'Drug':'SMILES'}, inplace =  True)
        normalized_smiles_list = normalize_smiles_parallel(data.SMILES.tolist())
        data["Normalized_SMILES"] = normalized_smiles_list
        data = data[~data.Normalized_SMILES.isnull()]
        data.to_csv(normlization_dir + f"{dataset}_seed_{seed}.csv", index = False)

Processing: 100%|█████████████████████████████| 48/48 [00:00<00:00, 2459.19it/s]


Processing: 100%|█████████████████████████████| 96/96 [00:00<00:00, 3649.60it/s]


# Get BERT representation
### change env to molbert

In [1]:
import os, yaml
import pandas as pd
from tqdm import tqdm
from molbert.models.smiles import SmilesMolbertModel

In [2]:
import logging
from typing import Tuple, Sequence, Any, Dict, Union, Optional

import numpy as np
import torch

from molbert.utils.featurizer.molfeaturizer import SmilesIndexFeaturizer

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

class MolBertFeaturizer:
    """
    This featurizer takes a molbert model and transforms the input data and
    returns the representation in the last layer (pooled output and sequence_output).
    """

    def __init__(
        self,
        model,
        featurizer,
        device: str = None,
        embedding_type: str = 'pooled',
        max_seq_len: Optional[int] = None,
        permute: bool = False,
    ) -> None:
        """
        Args:
            checkpoint_path: path or S3 location of trained model checkpoint
            device: device for torch
            embedding_type: method to reduce MolBERT encoding to an output set of features. Default: 'pooled'
                Other options are embeddings summed or concat across layers, and then averaged
                Raw sequence and pooled output is also available (set to 'dict')
                average-sum-[2|4], average-cat-[2,4], average-[1|2|3|4], average-1-cat-pooled, pooled, dict
            max_seq_len: used by the tokenizer, SMILES longer than this will fail to featurize
                MolBERT was trained with SuperPositionalEncodings (TransformerXL) to decoupled from the training setup
                By default the training config is used (128). If you have long SMILES to featurize, increase this value
        """
        super().__init__()
        self.device = device or 'cuda' if torch.cuda.is_available() else 'cpu'
        self.embedding_type = embedding_type
        self.max_seq_len = max_seq_len
        self.permute = permute

        # load smiles index featurizer
        self.featurizer = featurizer

        # load model
        self.model = model

    def __getstate__(self):
        self.__dict__.update({'model': self.model.to('cpu')})
        self.__dict__.update({'device': 'cpu'})
        return self.__dict__

    @property

    def transform_single(self, smiles: str) -> Tuple[np.ndarray, bool]:
        features, valid = self.transform([smiles])
        return features, valid[0]

    def transform(self, molecules: Sequence[Any]) -> Tuple[Union[Dict, np.ndarray], np.ndarray]:
        input_ids, valid = self.featurizer.transform(molecules)

        input_ids = self.trim_batch(input_ids, valid)

        token_type_ids = np.zeros_like(input_ids, dtype=np.int64)
        attention_mask = np.zeros_like(input_ids, dtype=np.int64)

        attention_mask[input_ids != 0] = 1

        input_ids = torch.tensor(input_ids, dtype=torch.long, device=self.device)
        token_type_ids = torch.tensor(token_type_ids, dtype=torch.long, device=self.device)
        attention_mask = torch.tensor(attention_mask, dtype=torch.long, device=self.device)

        with torch.no_grad():
            outputs = self.model.model.bert(
                input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
            )

        sequence_output, pooled_output = outputs

        # set invalid outputs to 0s
        valid_tensor = torch.tensor(
            valid, dtype=sequence_output.dtype, device=sequence_output.device, requires_grad=False
        )

        pooled_output = pooled_output * valid_tensor[:, None]
        sequence_out = sequence_output * valid_tensor[:, None, None]

        sequence_out = sequence_out.detach().cpu().numpy()
        pooled_output = pooled_output.detach().cpu().numpy()
        out = pooled_output

        return out, valid

    @staticmethod
    def trim_batch(input_ids, valid):

        # trim input horizontally if there is at least 1 valid data point
        if any(valid):
            _, cols = np.where(input_ids[valid] != 0)
        # else trim input down to 1 column (avoids empty batch error)
        else:
            cols = np.array([0])

        max_idx: int = int(cols.max().item() + 1)

        input_ids = input_ids[:, :max_idx]

        return input_ids

In [3]:
# config_dict
model_weights_dir = '/projects/home/mmasood1/Model_weights/invitro/invitro_1million/MolBERT/Retrain_on_top_of_BERT/complete_1m_300k_ADME/without_physchem_head/'
pretrained_model_path = '/projects/home/mmasood1/TG GATE/MolBERT/molbert/molbert_100epochs/molbert_100epochs/checkpoints/last.ckpt'
data_dir = '/projects/home/mmasood1/arslan_data_repository/invitro/invitro_1m/25_04_2024/SMILES_len_th_128/'
pos_weights = "/projects/home/mmasood1/arslan_data_repository/invitro/invitro_1m/25_04_2024/pos_weights.csv"
metadata_dir = "/projects/home/mmasood1/trained_model_predictions/SIDER_PreClinical/BERT_finetune/MF/"
model_dir = os.path.dirname(os.path.dirname(pretrained_model_path))
hparams_path = os.path.join(model_dir, 'hparams.yaml')
# load config
with open(hparams_path) as yaml_file:
    config_dict = yaml.load(yaml_file, Loader=yaml.FullLoader)

config_dict['project_name'] = "BERT_invitro_ADME_pretraining"
config_dict['model_name'] = "SMILES_len_th_128_Permute_False_PhySchem_False"

config_dict['model_weights_dir'] = model_weights_dir
config_dict['pretrained_model_path'] = pretrained_model_path
config_dict["metadata_dir"] = metadata_dir
config_dict['pos_weights'] = pos_weights
config_dict['data_dir'] = data_dir
config_dict['train_file'] = data_dir + "train_set_invitro_1m_300k_ADME_filtered.pkl"
config_dict['valid_file'] = data_dir + "test_set_invitro_1m_300k_ADME_filtered.pkl"
config_dict['test_file'] = data_dir + "test_set_invitro_1m_300k_ADME_filtered.pkl"

config_dict['mode'] = 'classification'
config_dict['alpha'] = 0.0
config_dict['beta'] = 0.0
config_dict['gamma'] = 0.0

config_dict['max_epochs'] = 50
config_dict['unfreeze_epoch'] = 210
config_dict["l2_lambda"] = 0.0
config_dict['embedding_size'] = 50
config_dict["num_physchem_properties"] = 200
config_dict["num_invivo_tasks"] = 0

config_dict['optim'] = 'AdamW'#SGD
config_dict['loss_type'] = 'BCE'# Focal_loss

config_dict['lr'] = 1e-05
config_dict["BERT_lr"] = 3e-5
config_dict["batch_size"] = 264
config_dict["seed"] = 42



config_dict['missing'] = 'nan'
config_dict['compute_metric_after_n_epochs'] = 5
config_dict['return_trainer'] = True
config_dict['EarlyStopping'] = False

config_dict["accelerator"] = "gpu"
config_dict["device"] = torch.device("cuda")


data = pd.read_pickle(config_dict['train_file'])
data.drop(['SMILES'], axis = 1, inplace = True)
target_names = data.columns.tolist()

config_dict["output_size"] = len(target_names)
config_dict["num_invitro_tasks"] = len(target_names)
config_dict["num_of_tasks"] = len(target_names)

config_dict["label_column"] = target_names
config_dict["selected_tasks"] = target_names
config_dict['num_mols'] = data.shape[0]
config_dict['max_seq_length'] = 128
config_dict['bert_output_dim'] = 768
config_dict['invitro_head_hidden_layer'] = 2048

config_dict["permute"] = False

config_dict['pretrained_model'] = True
config_dict['freeze_level'] = False
config_dict["gpu"] = -1
config_dict["precision"] = 32
config_dict["distributed_backend"] = "dp"
config_dict["pretrained_crash_model"] = None #"/projects/home/mmasood1/Model_weights/invitro/invitro_1million/MolBERT/Retrain_on_top_of_BERT/complete_1m_300k/invitro_with_PhysChem/epoch=2-val_f1_score=0.00.ckpt"

In [4]:
model = SmilesMolbertModel(config_dict)
checkpoint = torch.load(config_dict["pretrained_model_path"], map_location=lambda storage, loc: storage)
model.load_state_dict(checkpoint['state_dict'], strict = False)
model.eval()
model.freeze()
model = model.to("cpu")

featurizer = SmilesIndexFeaturizer.bert_smiles_index_featurizer(126, permute = False)
f = MolBertFeaturizer(model = model,
                        featurizer= featurizer,
                        device = "cpu")

42


In [5]:
def get_BERT_features(data):
    SMILES = data.Normalized_SMILES.tolist()
    features_all, masks_all = [],[]
    for s in tqdm(SMILES):
        features, masks = f.transform([s])
        features_all.append(features.squeeze())
        masks_all.append(masks)

    filtered = [mask[0] for mask in masks_all]
    features = pd.DataFrame(features_all)
    features = features[filtered]

    selected_SMILES = data[filtered].Normalized_SMILES.values
    features.insert(0,"SMILES", selected_SMILES)
    filtered_data = data[data.Normalized_SMILES.isin(selected_SMILES)].reset_index(drop = True)
    filtered_data.drop("SMILES", axis = 1, inplace = True)
    filtered_data.rename(columns = {"Normalized_SMILES":"SMILES"}, inplace = True)
    return filtered_data, features

In [6]:
data_dir = "/projects/home/mmasood1/arslan_data_repository/DILI"
dataset_list = ["train","valid","test"]

for seed in [1, 2, 3, 4, 5]:
    for dataset in dataset_list:

        # relevent dirs
        seed_dir = data_dir + f"/seed_{seed}/"
        normlization_dir = seed_dir + "normalized_data/"
        filtered_data_dir = normlization_dir + "filtered_data/"

        os.makedirs(filtered_data_dir, exist_ok = True)

        data = pd.read_csv(normlization_dir + f"{dataset}_seed_{seed}.csv")
        total_mol = data.shape[0]

        filtered_data, features = get_BERT_features(data)
        filtered_mol = filtered_data.shape[0]
        removed_mols = total_mol - filtered_mol
        print(seed, dataset, removed_mols)
        filtered_data.to_csv(filtered_data_dir + f"{dataset}_filtered_seed_{seed}.csv", index = False)
        features.to_csv(filtered_data_dir + f"{dataset}_DILI_BERT_features_seed_{seed}.csv", index = False)

  0%|          | 1/322 [00:00<01:39,  3.21it/s]

100%|██████████| 322/322 [00:51<00:00,  6.31it/s]


1 train 8


100%|██████████| 54/54 [00:06<00:00,  8.17it/s]


1 valid 0


...

5 train 7


100%|██████████| 48/48 [00:08<00:00,  5.59it/s]


5 valid 1


100%|██████████| 96/96 [00:16<00:00,  5.77it/s]


5 test 2


In [8]:
# compute pos_weights
data_dir = "/projects/home/mmasood1/arslan_data_repository/DILI"

for seed in [1,2,3,4,5]:

    # relevent dirs
    seed_dir = data_dir + f"/seed_{seed}/"
    normlization_dir = seed_dir + "normalized_data/"
    filtered_data_dir = normlization_dir + "filtered_data/"

    train = pd.read_csv(filtered_data_dir + f"train_filtered_seed_{seed}.csv")
    valid = pd.read_csv(filtered_data_dir + f"valid_filtered_seed_{seed}.csv")
    test = pd.read_csv(filtered_data_dir + f"test_filtered_seed_{seed}.csv")
    all_data = pd.concat([train, valid, test], axis = 0)
    pos = (all_data.Y == 1).sum()
    neg = (all_data.Y == 0).sum()
    class_weight = neg/pos
    class_weight = pd.DataFrame({"Targets":"Y",
                "weights":[class_weight]})
    class_weight.to_csv(filtered_data_dir + f"class_weight_seed_{seed}.csv", index = False)

,Targets,weights
0,Y,0.982833


In [16]:
pd.read_csv("/projects/home/mmasood1/arslan_data_repository/Tox21/pos_weights.csv")

,Targets,weights
0,NR-AR,22.511327
1,NR-AR-LBD,27.514768
2,NR-AhR,7.527344
3,NR-Aromatase,18.403333
4,NR-ER,6.809584
5,NR-ER-LBD,18.871429
6,NR-PPAR-gamma,33.677419
7,SR-ARE,5.191083
8,SR-ATAD5,25.787879
9,SR-HSE,16.384409
